# Intro to Machine learning, Final Project

In this project, we are given the Enron dataset, and the goal is to identify the persons of interest in the Enron scandal. We are provided with financial features, like salary, stock options, etc... and some email features, detailing the exchanges between the collaborators. We first import the datasets. The cleaned datasets are saved as pickle files, and can be accessed directly from [here](#section3). 

1. [Importation of the data](#section1) 

2. [Data Cleaning and Outliers](#section2)

3. [Start Classification from cleaned data](#section3)

   3.1. [Feature engineering](#section3.1)
   
   3.2. [Dimensionality reduction](#section3.2)

4. [Classification](#section4)

    4.1. [Gaussian Naive Bayes](#section4.1)
    
    4.2. [Logistic Regression](#section4.2)
    
    4.3. [Decision Tree](#section4.3)
    
    4.4. [Random Forest](#section4.4)
    
    4.5. [AdaBoost](#section4.5)

## Importation of the data:
<a id='section1'></a>

In [106]:
import sys
import pickle
import numpy as np
with open("../ud120-projects/final_project/final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
import pandas as pd
data=pd.DataFrame.from_dict(data_dict, orient='index')
data.replace('NaN', np.nan, inplace=True) #replace the NaN values (strings) as np.nan values
print(data.head(2))
print('There are {} persons in this dataset, with {} features'.format(data.shape[0], data.shape[1]))
print('The features are: {}'.format(data.columns.values))

                   salary  to_messages  deferral_payments  total_payments  \
ALLEN PHILLIP K  201955.0       2902.0          2869717.0       4484442.0   
BADUM JAMES P         NaN          NaN           178980.0        182466.0   

                 exercised_stock_options      bonus  restricted_stock  \
ALLEN PHILLIP K                1729541.0  4175000.0          126027.0   
BADUM JAMES P                   257817.0        NaN               NaN   

                 shared_receipt_with_poi  restricted_stock_deferred  \
ALLEN PHILLIP K                   1407.0                  -126027.0   
BADUM JAMES P                        NaN                        NaN   

                 total_stock_value           ...            loan_advances  \
ALLEN PHILLIP K          1729541.0           ...                      NaN   
BADUM JAMES P             257817.0           ...                      NaN   

                 from_messages  other  from_this_person_to_poi    poi  \
ALLEN PHILLIP K         2195.

## Data cleaning and outliers
<a id='section2'></a>

We start with some data cleaning, identifying some outliers and absurdities. The first task is to count the number of 'NaN' values for each features.

In [107]:
nb_NaN=data.isnull().sum(axis=0)
print(nb_NaN)

salary                        51
to_messages                   60
deferral_payments            107
total_payments                21
exercised_stock_options       44
bonus                         64
restricted_stock              36
shared_receipt_with_poi       60
restricted_stock_deferred    128
total_stock_value             20
expenses                      51
loan_advances                142
from_messages                 60
other                         53
from_this_person_to_poi       60
poi                            0
director_fees                129
deferred_income               97
long_term_incentive           80
email_address                 35
from_poi_to_this_person       60
dtype: int64


We see that some of the features have a lot of missing values. Furthermore, some features are irrelevant, like the email address. We drop those features.

In [108]:
irrelevant_features=nb_NaN.index[nb_NaN>80].values
irrelevant_features=np.append(irrelevant_features,['email_address'])
data.drop(irrelevant_features,axis=1, inplace=True)
print(data.head(2))

                   salary  to_messages  total_payments  \
ALLEN PHILLIP K  201955.0       2902.0       4484442.0   
BADUM JAMES P         NaN          NaN        182466.0   

                 exercised_stock_options      bonus  restricted_stock  \
ALLEN PHILLIP K                1729541.0  4175000.0          126027.0   
BADUM JAMES P                   257817.0        NaN               NaN   

                 shared_receipt_with_poi  total_stock_value  expenses  \
ALLEN PHILLIP K                   1407.0          1729541.0   13868.0   
BADUM JAMES P                        NaN           257817.0    3486.0   

                 from_messages  other  from_this_person_to_poi    poi  \
ALLEN PHILLIP K         2195.0  152.0                     65.0  False   
BADUM JAMES P              NaN    NaN                      NaN  False   

                 long_term_incentive  from_poi_to_this_person  
ALLEN PHILLIP K             304805.0                     47.0  
BADUM JAMES P                    NaN 

The new feature list is given by:

In [109]:
print(data.columns.values)

['salary' 'to_messages' 'total_payments' 'exercised_stock_options' 'bonus'
 'restricted_stock' 'shared_receipt_with_poi' 'total_stock_value'
 'expenses' 'from_messages' 'other' 'from_this_person_to_poi' 'poi'
 'long_term_incentive' 'from_poi_to_this_person']


Next, we look for outliers in the data. To do this, we select the financial features and print the 5 largest values for each.

In [110]:
financial_features=['salary','total_payments' ,'exercised_stock_options', 'bonus',
 'restricted_stock' , 'total_stock_value',
 'expenses' , 'other' ,
 'long_term_incentive' ]
for feature in financial_features:
    print('Largest {}:'.format(feature))
    print(data.nlargest(5,feature)[feature])
    print('')

Largest salary:
TOTAL                 26704229.0
SKILLING JEFFREY K     1111258.0
LAY KENNETH L          1072321.0
FREVERT MARK A         1060932.0
PICKERING MARK R        655037.0
Name: salary, dtype: float64

Largest total_payments:
TOTAL               309886585.0
LAY KENNETH L       103559793.0
FREVERT MARK A       17252530.0
BHATNAGAR SANJAY     15456290.0
LAVORATO JOHN J      10425757.0
Name: total_payments, dtype: float64

Largest exercised_stock_options:
TOTAL                 311764000.0
LAY KENNETH L          34348384.0
HIRKO JOSEPH           30766064.0
RICE KENNETH D         19794175.0
SKILLING JEFFREY K     19250000.0
Name: exercised_stock_options, dtype: float64

Largest bonus:
TOTAL                 97343619.0
LAVORATO JOHN J        8000000.0
LAY KENNETH L          7000000.0
SKILLING JEFFREY K     5600000.0
BELDEN TIMOTHY N       5249999.0
Name: bonus, dtype: float64

Largest restricted_stock:
TOTAL                 130322299.0
LAY KENNETH L          14761694.0
WHITE JR THOMA

There is a 'TOTAL' index that is useless here, we drop it.

In [111]:
data.drop('TOTAL',axis=0,inplace=True)

We also check for indexes which have almost no data:

In [112]:
nb_NaN_index=data.isnull().sum(axis=1)
print(nb_NaN_index.nlargest(40))

LOCKHART EUGENE E                14
CHAN RONNIE                      13
GRAMM WENDY L                    13
SAVAGE FRANK                     13
BLAKE JR. NORMAN P               12
CLINE KENNETH W                  12
MENDELSOHN JOHN                  12
MEYER JEROME J                   12
PEREIRA PAULO V. FERRAZ          12
SCRIMSHAW MATTHEW                12
THE TRAVEL AGENCY IN THE PARK    12
URQUHART JOHN A                  12
WAKEHAM JOHN                     12
WHALEY DAVID A                   12
WINOKUR JR. HERBERT S            12
WODRASKA JOHN                    12
WROBEL BRUCE                     12
BELFER ROBERT                    11
CHRISTODOULOU DIOMEDES           11
DUNCAN JOHN H                    11
FUGH JOHN L                      11
GATHMANN WILLIAM D               11
GILLIS JOHN                      11
LEMAISTRE CHARLES                11
LOWRY CHARLES P                  11
NOLES JAMES L                    11
BADUM JAMES P                    10
GRAY RODNEY                 

We drop all the indices which have 12 or more features with no information (out of 15 features in total). In particular, looking at the list, we realize that there is a travel agency that does not have its place here.

In [113]:
irrelevant_indices=nb_NaN_index.index[nb_NaN_index>=12].values
data.drop(irrelevant_indices,axis=0, inplace=True)

We can now start working with a cleaner dataset, and we split the 'poi' target feature appart.

In [115]:
labels=data['poi']
features=data.drop('poi', axis=1)
print(labels.head())
print(features.head())
print(features.shape)

ALLEN PHILLIP K       False
BADUM JAMES P         False
BANNANTINE JAMES M    False
BAXTER JOHN C         False
BAY FRANKLIN R        False
Name: poi, dtype: bool
                      salary  to_messages  total_payments  \
ALLEN PHILLIP K     201955.0       2902.0       4484442.0   
BADUM JAMES P            NaN          NaN        182466.0   
BANNANTINE JAMES M     477.0        566.0        916197.0   
BAXTER JOHN C       267102.0          NaN       5634343.0   
BAY FRANKLIN R      239671.0          NaN        827696.0   

                    exercised_stock_options      bonus  restricted_stock  \
ALLEN PHILLIP K                   1729541.0  4175000.0          126027.0   
BADUM JAMES P                      257817.0        NaN               NaN   
BANNANTINE JAMES M                4046157.0        NaN         1757552.0   
BAXTER JOHN C                     6680544.0  1200000.0         3942714.0   
BAY FRANKLIN R                          NaN   400000.0          145796.0   

             

We save this clean dataset to pickle files.

In [116]:
labels.to_pickle('labels.pkl')
features.to_pickle('features.pkl')

## Start classification from cleaned data 
<a id='section3'></a>

We can start to train a classifier to the new data to try to identify persons of inerest.

In [115]:
import sys
import pickle
import numpy as np
import pandas as pd
labels=pd.read_pickle('labels.pkl')
features=pd.read_pickle('features.pkl')

### Feature engineering
<a id='section3.1'></a>

We recall the features that we have:

In [117]:
print(features.columns.values)

['salary' 'to_messages' 'total_payments' 'exercised_stock_options' 'bonus'
 'restricted_stock' 'shared_receipt_with_poi' 'total_stock_value'
 'expenses' 'from_messages' 'other' 'from_this_person_to_poi'
 'long_term_incentive' 'from_poi_to_this_person']


Some of those features do not seem to be particularly relevant for the problem at stake. For instance, the features 'to messages' and 'from message' are the total number of messages received and sent by a person. However, we can use those features to create a feature that stores the proportion of messages in relation with a POI. 

In [118]:

features['poi_index']=(features['from_this_person_to_poi']+features['from_poi_to_this_person']+
                                  features['shared_receipt_with_poi'])/(features['to_messages']+
                                                                        features['from_messages'])

features.drop(['to_messages','from_messages','from_poi_to_this_person','from_this_person_to_poi',
               'shared_receipt_with_poi'],axis=1,inplace=True)
print(features.columns.values)

['salary' 'total_payments' 'exercised_stock_options' 'bonus'
 'restricted_stock' 'total_stock_value' 'expenses' 'other'
 'long_term_incentive' 'poi_index']


Now, we standardize the financial features, to have all the numeric values on a similar scale.

In [119]:
financial_features=['salary' ,'total_payments' ,'exercised_stock_options' ,'bonus',
 'restricted_stock', 'total_stock_value', 'expenses' ,'other',
 'long_term_incentive']
for f in financial_features:
    features[f]=(features[f]-features[f].min())/(features[f].max()-features[f].min())

print(features.head())

                      salary  total_payments  exercised_stock_options  \
ALLEN PHILLIP K     0.181384        0.043299                 0.050262   
BADUM JAMES P            NaN        0.001757                 0.007411   
BANNANTINE JAMES M  0.000000        0.008842                 0.117713   
BAXTER JOHN C       0.240034        0.054402                 0.194417   
BAY FRANKLIN R      0.215339        0.007988                      NaN   

                       bonus  restricted_stock  total_stock_value  expenses  \
ALLEN PHILLIP K     0.517654          0.157232           0.036083  0.058667   
BADUM JAMES P            NaN               NaN           0.006142  0.013189   
BANNANTINE JAMES M       NaN          0.251180           0.107571  0.244542   
BAXTER JOHN C       0.142497          0.377009           0.217018  0.046980   
BAY FRANKLIN R      0.041614          0.158370           0.002179  0.563617   

                       other  long_term_incentive  poi_index  
ALLEN PHILLIP K     0.0

There are still many NaN values in this dataset, and we will replace those by 0. Indeed, for the features in consideration, in the absence of any information it makes sense to set those values to 0, instead of the mean or the median for example. Suppressing these rows would create a much smaller dataset.

In [120]:
features.fillna(0.,inplace=True)

### Dimensionality reduction
<a id='section3.2'></a>

The financial features are certainly highly correlated, we wil perform a PCA on those features.

In [24]:
from sklearn.decomposition import PCA
pca=PCA()

pca.fit(features)

print(pca.explained_variance_ratio_)

[  4.70845380e-01   2.55629689e-01   9.58668143e-02   5.77620576e-02
   3.68834174e-02   3.51819187e-02   2.49955042e-02   1.86671866e-02
   3.75754433e-03   4.10487907e-04]


The last two features of the pca explain less than 1% of the variance. We will therefore perform dimensionality reduction and suppress those 2 features fr our analysis. The PCA step will also be needed for testing, it will be included later in a Pipeline.

In [25]:
pca=PCA(n_components=8)
pca.fit(features)

PCA(copy=True, iterated_power='auto', n_components=8, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

## Classification
<a id='section4'></a>

We now try different classifyers, from very simple ones like naive bayes, to ensemble methods with gradient boosting. The assesment of these performance will be done vie 10-fold cross validation.

In [61]:
def classification_report(clf, features, labels, nfolds):
    from sklearn.model_selection import StratifiedKFold
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    skf=StratifiedKFold(n_splits=nfolds)
    scores=[(0,0,0,0)]*nfolds
    i=0
    for train_index, test_index in skf.split(features, labels):
        features_train=features.iloc[train_index]
        features_test=features.iloc[test_index]
        labels_train=labels[train_index]
        labels_test=labels[test_index]
        clf.fit(features_train,labels_train)
        pred=clf.predict(features_test)
        precision,recall,f_score,_ =precision_recall_fscore_support(labels_test,pred, average='binary')
        acc=accuracy_score(labels_test,pred)
        scores[i]=(acc,precision,recall,f_score)
        i=i+1
    print('Accuracy: {}   ,   Precision: {}   ,   Recall: {}   ,   F1 score: {}'.format(*np.mean(scores,axis=0)))
    return np.mean(scores,axis=0)

### Gaussian Naive Bayes
<a id='section4.1'></a>

With PCA

In [62]:
nfolds=10
from sklearn.naive_bayes import GaussianNB
GNB=GaussianNB()

from sklearn.pipeline import Pipeline
pipe=Pipeline([('reduce_dim',PCA(n_components=8)), ('naive_bayes',GNB)])

classification_report(pipe,features,labels, nfolds)

Accuracy: 0.821794871795   ,   Precision: 0.29   ,   Recall: 0.3   ,   F1 score: 0.27380952381


array([ 0.82179487,  0.29      ,  0.3       ,  0.27380952])

Without PCA:

In [63]:
nfolds=10
from sklearn.naive_bayes import GaussianNB
GNB=GaussianNB()

classification_report(GNB,features,labels, nfolds)

Accuracy: 0.821794871795   ,   Precision: 0.233333333333   ,   Recall: 0.2   ,   F1 score: 0.206666666667


array([ 0.82179487,  0.23333333,  0.2       ,  0.20666667])

### Logistic regression
<a id='section4.2'></a>

With PCA:

In [65]:
nfolds=10
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()

from sklearn.pipeline import Pipeline
pipe=Pipeline([('reduce_dim',PCA(n_components=8)), ('Log_reg',clf)])

classification_report(pipe,features,labels, nfolds)

Accuracy: 0.86858974359   ,   Precision: 0.2   ,   Recall: 0.15   ,   F1 score: 0.166666666667


array([ 0.86858974,  0.2       ,  0.15      ,  0.16666667])

Without PCA:

In [66]:
nfolds=10
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()

classification_report(clf,features,labels, nfolds)

Accuracy: 0.867948717949   ,   Precision: 0.1   ,   Recall: 0.05   ,   F1 score: 0.0666666666667


array([ 0.86794872,  0.1       ,  0.05      ,  0.06666667])

Let's try to improve a little bit this simple classifier using some regularization:

In [71]:
nfolds=10
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()

from sklearn.pipeline import Pipeline
pipe=Pipeline([('reduce_dim',PCA(n_components=8)), ('Log_reg',clf)])

from sklearn.model_selection import GridSearchCV
C = np.logspace(-4, 4, 50)
penalty = ['l1', 'l2']

parameters = dict(Log_reg__C=C,
                  Log_reg__penalty=penalty)

clf = GridSearchCV(pipe, parameters,n_jobs=-1, cv=10)


clf.fit(features, labels)

best_clf=clf.best_estimator_

classification_report(best_clf,features,labels, nfolds)

Accuracy: 0.876282051282   ,   Precision: 0.2   ,   Recall: 0.2   ,   F1 score: 0.2


array([ 0.87628205,  0.2       ,  0.2       ,  0.2       ])

These scores are not very good, we shall try more sophisticated methods. Let's first try a simple Decision tree.

### Decision Tree
<a id='section4.3'></a>

With PCA:

In [82]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score
from sklearn.metrics import make_scorer
clf=DecisionTreeClassifier()

from sklearn.pipeline import Pipeline
pipe=Pipeline([('reduce_dim',PCA(n_components=8)), ('decision_tree',clf)])

from sklearn.model_selection import GridSearchCV

parameters = dict(decision_tree__criterion=['gini', 'entropy'],
                  decision_tree__max_features=[5,6,7,8], decision_tree__max_depth=[None,3,5,10], 
                 decision_tree__min_samples_split =[2,5], decision_tree__class_weight=[None, 'balanced'])

clf = GridSearchCV(pipe, parameters,n_jobs=-1, cv=10, scoring=make_scorer(f1_score))

clf.fit(features, labels)

best_clf=clf.best_estimator_

classification_report(best_clf,features,labels, nfolds)

/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 0.598076923077   ,   Precision: 0.240952380952   ,   Recall: 0.6   ,   F1 score: 0.329365079365


array([ 0.59807692,  0.24095238,  0.6       ,  0.32936508])

Good recall but still poor precision, therefore we try some ensemble method:

### Random Forest
<a id='section4.4'></a>

In [85]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score
from sklearn.metrics import make_scorer


from sklearn.pipeline import Pipeline
pipe=Pipeline([('reduce_dim',PCA(n_components=8)), ('r_forest',RandomForestClassifier())])

from sklearn.model_selection import GridSearchCV

parameters = dict(r_forest__criterion=['gini', 'entropy'], r_forest__n_estimators=[5,10],
                  r_forest__max_features=[5,6,7,8], r_forest__max_depth=[None,3,5,10], 
                 r_forest__min_samples_split =[2,5], r_forest__class_weight=[None, 'balanced'], r_forest__n_jobs=[-1])

clf = GridSearchCV(pipe, parameters,n_jobs=-1, cv=10, scoring=make_scorer(precision_score))

clf.fit(features, labels)

best_clf=clf.best_estimator_

classification_report(best_clf,features,labels, nfolds)

/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 0.75   ,   Precision: 0.15   ,   Recall: 0.3   ,   F1 score: 0.188333333333


array([ 0.75      ,  0.15      ,  0.3       ,  0.18833333])

### AdaBoost
<a id='section4.5'></a>

In [121]:
nfolds=10
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier()

lr=[0.01, 0.05, 0.1, 0.5, 1., 2., 5., 10.] 

parameters=dict(adaboost__learning_rate=lr, adaboost__n_estimators=[10,50,100])

from sklearn.pipeline import Pipeline
pipe=Pipeline([('adaboost',ada)])

clf = GridSearchCV(pipe, parameters,n_jobs=-1, cv=5, scoring=make_scorer(f1_score))

clf.fit(features, labels)

best_clf=clf.best_estimator_

classification_report(best_clf,features,labels, nfolds)


/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/thumeau/anaconda/envs/py2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy: 0.860256410256   ,   Precision: 0.316666666667   ,   Recall: 0.35   ,   F1 score: 0.33


array([ 0.86025641,  0.31666667,  0.35      ,  0.33      ])